# Facebook Stock Event Study

*A few weeks ago Facebook's stock skyrocketed after they announced the fourth-quarter earnings on the 30th of January 2019. Despite all privacy scandals and criticism in the media recently, the tech unicorn proved investors wrong: stock prices exceeded the 170 dollar mark after a long period of decline. Figure 1 illustrates the financial consequences of this event as compared to overall industry market growth (NASDAQ).*

*In this assignment we apply the event study method to demonstrate that stock returns on the 30th and 31th of January can be regarded as abnormal. This method assumes that * 

![Facebook Stock](FB_stock.png)
**Figure 1**: Facebook (dark blue) vs NASDAQ (light blue) stock price growth


In [1]:
# collect Facebook stock data for the last 2 years (IEX Trading API)
import requests, pandas as pd

def get_stock(stock_symbol):
    r = requests.get('https://api.iextrading.com/1.0/stock/' + stock_symbol + '/chart/2y')
    result = r.json()
    return pd.DataFrame(list({pd.to_datetime(date['date']): date['close'] for date in result}.items()), columns=['date', 'stock']) 

fb = get_stock('fb')
ndaq = get_stock('ndaq')

In [ ]:
# define stock returns
def stock_returns(df): 
    for row in range(1, len(df)):
        df.loc[row, "stock_return"] = (df.loc[row, "stock"] - df.loc[row-1, "stock"]) / df.loc[row-1, "stock"]
        df.loc[row, "actual_stock_return"] = (df.loc[row, "stock"] - df.loc[row-1, "stock"]) - 1
    return df

fb = stock_returns(fb)
ndaq = stock_returns(ndaq)

In [ ]:
# run simple ols model on stock returns (14th of February 2017 to 29th of January 2019)
import statsmodels.api as sm
from scipy import stats
end_date = '2019-01-29'

X = ndaq.loc[1:fb[fb.date == end_date].index[0], ['stock_return']] 
y = fb.loc[1:fb[fb.date == end_date].index[0], ['stock_return']]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2.astype(float))
est2 = est.fit()
print(est2.summary())

In [ ]:
# calculate expected Facebook returns
df_merged = pd.merge(fb, ndaq, left_on = 'date', right_on = 'date', suffixes=('_fb', '_ndaq'))
df_merged['expected_return_fb'] = est2.params[0] + est2.params[1] * df_merged['actual_stock_return_ndaq']
df_merged['abnormal_return_fb'] = df_merged['actual_stock_return_fb'] - df_merged['expected_return_fb']

In [ ]:
# top 5 dates positive abnormal returns 
df_merged[['date', 'abnormal_return_fb']].sort_values(by='abnormal_return_fb', ascending=False).head()
# note that the 31st of January 2019 has the highest abnormal return!

In [ ]:
# top 5 dates negative abnormal returns 
df_merged[['date', 'abnormal_return_fb']].sort_values(by='abnormal_return_fb', ascending=True).head()
# this CNBC article explains why the 26th of June was Facebook's worst day ever: https://www.cnbc.com/2018/07/26/facebook-is-on-pace-for-its-worst-day-ever.html

In [ ]:
# a more detailed looks suggests a 2-day window would be most appropriate (30th and 31st of January)
df_merged.loc[491:498, ['date', 'abnormal_return_fb']]

In [ ]:
# calculate cumulative abornomal returns (CAR)
CAR = df_merged.loc[df_merged.date == '2019-01-30', 'abnormal_return_fb'].item() + df_merged.loc[df_merged.date == '2019-01-31', 'abnormal_return_fb'].item()

In [ ]:
# calculate standardized abnormal returns (SCAR)
import numpy as np

event_window_length = 2
var_e = np.var(est2.resid)

SCAR = (CAR/100) / (event_window_length * var_e)**.5

print("All in all, we can conclude that abnormal returns in event period are " + str(round(SCAR,1)) + " standard deviations from estimation period abnormal returns. Hence, the publication of fourth-quarter results on January 30th 2019 had a significant positive effect on Facebook's stock price.")